In [1]:
import os
import sys
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from modules import *
import json

In [2]:
gpt2 = {'l': 1024, 'e': 1600, 'h': 32, 'depth': 48}
gpt3 = {'l': 2048, 'e': 12288, 'h': 96, 'depth': 96}
gpt3_1T = {'l': 2048, 'e': 25600, 'h': 160, 'depth': 180}
gpt3_lowdepth = {'l': 2048, 'e': 12288, 'h': 96, 'depth': 96 // 8}
vit_era5 = {'l': 64800, 'e': 5120, 'h': 256, 'depth': 24}
vit_era5_big = {'l': 64800, 'e': 6144, 'h': 32, 'depth': 32}
### model
model = gpt3_1T
b = 1
l = model['l']
e = model['e']
f = 4 * e
h = model['h']
depth = model['depth']
print("model: batch size = {}, seq length = {}, embed = {}, attention heads = {}, depth = {}".format(b, l, e, h, depth))

model: batch size = 1, seq length = 2048, embed = 25600, attention heads = 160, depth = 180


In [3]:
with open('config.json', 'r') as file:
    system = json.load(file)

In [4]:
def print_df(df_mlp, df_sa):
    cols = df_mlp.columns.tolist()
    layer_track_cols = ['activation_buffer', 'weights_mem', 
                        'weights_grad_mem', 'flops_fwd', 'flops_bwd', 
                        't_fwd', 't_fwd_comm', 't_bwd', 't_bwd_comm']
    display(df_mlp[cols])
    display(df_mlp[layer_track_cols].sum() * depth)
    display(df_sa[cols])
    display(df_sa[layer_track_cols].sum() * depth)
    t_itr = (df_mlp['t_fwd'].sum() + df_mlp['t_bwd'].sum() + df_sa['t_fwd'].sum() + df_sa['t_bwd'].sum()) * depth
    print('time for 1 itr = {}'.format(t_itr))

    f1 = 3 # 1 fp16 wt, 1 fp32 copy
    f2 = 5 # 1 fp16 grad, 2 fp32 means and variances
    mem = (df_mlp['weights_mem'].sum() * f1 + df_mlp['weights_grad_mem'].sum() * f2 + df_mlp['activation_buffer'].sum() +
           df_sa['weights_mem'].sum() * f1 + df_sa['weights_grad_mem'].sum() * f2 + df_sa['activation_buffer'].sum()) * depth
    print('mem consumed = {}'.format(mem))

In [6]:
m1 = 32
system['nvlink_size'] = 4
t1 = m1 if m1 <= system['nvlink_size'] else system['nvlink_size']
df_mlp = mlp_1d(b, l, e, f, parallelism={'m': m1}, topology={'t': t1}, system=system)
df_sa = sa_1d(b, l, e, h, parallelism={'m': m1}, topology={'t': t1}, flash_attention=True, system=system)
print_df(df_mlp, df_sa)

,name,weights_mem,weights_grad_mem,flops_fwd,activation_buffer,comm_fwd,comm_fwd_type,comm_fwd_size,comm_fwd_topology,flops_bwd,comm_bwd,comm_bwd_type,comm_bwd_size,comm_bwd_topology,t_fwd,t_fwd_comm,intensity_fwd,t_bwd,t_bwd_comm,intensity_bwd,t
0,fc1,0.163840,0.163840,0.335538,0.104858,0.000000,reducescatter,1,none,0.670954,0.104858,reducescatter,32,4,0.001075,0.000000,5.934291,0.003407,0.001256,5.933219,0.004482
1,fc1-bias,0.000006,0.000006,0.000007,0.000000,0.000000,none,0,none,0.000007,0.000000,allreduce,1,none,0.000008,0.000000,0.009963,0.000008,0.000000,0.009963,0.000017
2,act1,0.000000,0.000000,0.000007,0.013107,0.000000,none,0,none,0.000007,0.000000,none,0,none,0.000017,0.000000,0.004984,0.000025,0.000000,0.003323,0.000042
3,dpr1,0.000000,0.000000,0.000007,0.006554,0.000000,none,0,none,0.000007,0.000000,none,0,none,0.000021,0.000000,0.003987,0.000021,0.000000,0.003987,0.000042
4,fc2,0.163840,0.163840,0.335492,0.013107,0.104858,reducescatter,32,4,0.671000,0.000000,reducescatter,1,none,0.002331,0.001256,5.933479,0.002151,0.000000,5.933624,0.004482
5,fc2-bias,0.000051,0.000051,0.000052,0.000000,0.000000,none,0,none,0.000052,0.000000,allreduce,1,none,0.000067,0.000000,0.009963,0.000067,0.000000,0.009963,0.000135
6,dpr2,0.000000,0.000000,0.000002,0.001638,0.000000,none,0,none,0.000002,0.000000,none,0,none,0.000005,0.000000,0.003987,0.000005,0.000000,0.003987,0.000011
7,ln1,0.000102,0.000102,0.000015,0.003277,0.104858,allgather,32,4,0.000020,0.104858,reducescatter,32,4,0.001260,0.001256,0.044164,0.001267,0.001256,0.023774,0.002527


activation_buffer     25.657344
weights_mem           59.011200
weights_grad_mem      59.011200
flops_fwd            120.801264
flops_bwd            241.568612
t_fwd                  0.861539
t_fwd_comm             0.452198
t_bwd                  1.251266
t_bwd_comm             0.452198
dtype: float64

,name,weights_mem,weights_grad_mem,flops_fwd,activation_buffer,comm_fwd,comm_fwd_type,comm_fwd_size,comm_fwd_topology,flops_bwd,comm_bwd,comm_bwd_type,comm_bwd_size,comm_bwd_topology,t_fwd,t_fwd_comm,intensity_fwd,t_bwd,t_bwd_comm,intensity_bwd,t
0,qkv,0.122880,0.122880,0.251653,0.104858,0.000000,reducescatter,1,none,0.503203,0.104858,reducescatter,32,4,0.000807,0.000000,5.279472,0.002869,0.001256,5.278381,0.003676
1,fusedla,0.000000,0.000000,0.013735,0.013148,0.000000,none,0,none,0.034262,0.000000,none,0,none,0.000044,0.000000,5.214419,0.000110,0.000000,6.503939,0.000154
2,vproj,0.040960,0.040960,0.083834,0.003277,0.104858,reducescatter,32,4,0.167750,0.000000,reducescatter,1,none,0.001525,0.001256,2.802418,0.000538,0.000000,2.803800,0.002062
3,vproj-bias,0.000051,0.000051,0.000052,0.000000,0.000000,none,0,none,0.000052,0.000000,allreduce,1,none,0.000067,0.000000,0.009963,0.000067,0.000000,0.009963,0.000135
4,dpr_v,0.000000,0.000000,0.000002,0.001638,0.000000,none,0,none,0.000002,0.000000,none,0,none,0.000005,0.000000,0.003987,0.000005,0.000000,0.003987,0.000011
5,ln2,0.000102,0.000102,0.000015,0.003277,0.104858,allgather,32,4,0.000020,0.104858,reducescatter,32,4,0.001260,0.001256,0.044164,0.001267,0.001256,0.023774,0.002527


activation_buffer     22.715597
weights_mem           29.518848
weights_grad_mem      29.518848
flops_fwd             62.872282
flops_bwd            126.951992
t_fwd                  0.667535
t_fwd_comm             0.452198
t_bwd                  0.874054
t_bwd_comm             0.452198
dtype: float64

time for 1 itr = 3.654393759374722
mem consumed = 756.6133248


In [ ]:
m1 = 4
m2 = 4
t1 = 4
t2 = 1
system['nvlink_size'] = 4
system['summa_nb'] = 16
df_mlp = mlp_2d(b, l, e, f, parallelism={'m1': m1, 'm2': m2}, topology={'t1': t1, 't2': t2}, system=system)
df_sa = sa_2d_seqp(b, l, e, h, parallelism={'m1': m1, 'm2': m2}, topology={'t1': t1, 't2': t2}, flash_attention=True, system=system)
print_df(df_mlp, df_sa)

In [ ]:
def plot(n_gpus, system, axs, lgnd=['MLP', 'SA'], lgnd_tot=['nvlink1'], lfmt="-"):
    t_mlp = []
    t_sa = []
    t_itr = []

    for n in n_gpus:
        m1 = n
        t1 = m1 if m1 <= system['nvlink_size'] else system['nvlink_size']
        
        df_mlp = mlp_1d(b, l, e, f, parallelism={'m': m1}, topology={'t': t1}, system=system)
        df_sa = sa_1d(b, l, e, h, parallelism={'m': m1}, topology={'t': t1}, flash_attention=True, system=system)

        t_mlp_ = (df_mlp['t_fwd'].sum() + df_mlp['t_bwd'].sum()) * depth
        t_sa_ = (df_sa['t_fwd'].sum() + df_sa['t_bwd'].sum()) * depth
        t_itr.append(t_mlp_ + t_sa_)
        t_mlp.append(t_mlp_)
        t_sa.append(t_sa_)

    
    ax = axs[0]
    ax.plot(n_gpus, t_mlp, lfmt, linewidth=2, c=c1)
    ax.plot(n_gpus, t_sa, lfmt, linewidth=2, c=c2)
    ax.set_yscale('log')
    ax.set_xscale('log', base=2)
    ax.set_xlabel('Number of GPUs', fontsize=fsz)
    ax.set_xticks(n_gpus)
    ax.set_xticklabels(n_gpus, fontsize=fsz-4)
    ax.set_ylabel('Time', fontsize=fsz)    
    ax.legend(lgnd, fontsize=fsz-4)
    
    ax = axs[1]
    ax.plot(n_gpus, t_itr, lfmt, linewidth=2)
    ax.set_yscale('log')
    ax.set_xscale('log', base=2)
    ax.set_xlabel('Number of GPUs', fontsize=fsz)
    ax.set_xticks(n_gpus)
    ax.set_xticklabels(n_gpus, fontsize=fsz-4)
    ax.set_ylabel('Total time', fontsize=fsz)
    ax.legend(lgnd_tot, fontsize=fsz-4)
    ax.yaxis.set_minor_formatter(FormatStrFormatter("%d"))

In [ ]:
# Sweeps
### model parallelism
n_gpus = [1, 4, 8, 16, 32, 64, 128]
with open('config.json', 'r') as file:
    system = json.load(file)
fig, axs = plt.subplots(1,2,figsize=(10,5), tight_layout=True) 
c1 = 'steelblue'
c2 = 'salmon'
fsz = 18

nvs = 4
system['nvlink_size'] = nvs
lgnd = ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot = ["nvlink{}".format(nvs)]
plot(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="o-")
nvs = 16
system['nvlink_size'] = nvs
lgnd += ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot += ["nvlink{}".format(nvs)]
plot(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="*:")
nvs = 2
system['nvlink_size'] = nvs
lgnd += ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot += ["nvlink{}".format(nvs)]
plot(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="o--")

In [ ]:
def set_gpus(n, nvs):
    parallelism = {}
    topology = {}
    factors_of_n = [[i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0]
    best_factor = factors_of_n[-1]
    parallelism['m1'] = best_factor[0]
    parallelism['m2'] = best_factor[1]
    
    m1 = parallelism['m1'] 
    m2 = parallelism['m2'] 
    topology['t1'] = m1 if m1 <= system['nvlink_size'] else system['nvlink_size']
    topology['t2'] = 1
        
    # print(parallelism, topology)
    return parallelism, topology

def plot_2d(n_gpus, system, axs, lgnd=['MLP', 'SA'], lgnd_tot=['nvlink1'], lfmt="-"):
    t_mlp = []
    t_sa = []
    t_itr = []

    for n in n_gpus:
        parallelism, topology = set_gpus(n, system['nvlink_size'])
        
        df_mlp = mlp_2d(b, l, e, f, parallelism=parallelism, topology=topology, system=system)
        df_sa = sa_2d_seqp(b, l, e, h, parallelism=parallelism, topology=topology, flash_attention=True, system=system)
        # df_sa = sa_2d(b, l, e, h, parallelism=parallelism, topology=topology, system=system)

        t_mlp_ = (df_mlp['t_fwd'].sum() + df_mlp['t_bwd'].sum()) * depth
        t_sa_ = (df_sa['t_fwd'].sum() + df_sa['t_bwd'].sum()) * depth
        t_itr.append(t_mlp_ + t_sa_)
        t_mlp.append(t_mlp_)
        t_sa.append(t_sa_)

    
    ax = axs[0]
    ax.plot(n_gpus, t_mlp, lfmt, linewidth=2, c=c1)
    ax.plot(n_gpus, t_sa, lfmt, linewidth=2, c=c2)
    ax.set_yscale('log')
    ax.set_xscale('log', base=2)
    ax.set_xlabel('Number of GPUs', fontsize=fsz)
    ax.set_xticks(n_gpus)
    ax.set_xticklabels(n_gpus, fontsize=fsz-4)
    ax.set_ylabel('Time', fontsize=fsz)    
    ax.legend(lgnd, fontsize=fsz-4)
    
    ax = axs[1]
    ax.plot(n_gpus, t_itr, lfmt, linewidth=2)
    ax.set_yscale('log')
    ax.set_xscale('log', base=2)
    ax.set_xlabel('Number of GPUs', fontsize=fsz)
    ax.set_xticks(n_gpus)
    ax.set_xticklabels(n_gpus, fontsize=fsz-4)
    ax.set_ylabel('Total time', fontsize=fsz)
    ax.legend(lgnd_tot, fontsize=fsz-4)
    ax.yaxis.set_minor_formatter(FormatStrFormatter("%d"))

In [ ]:
# Sweeps
### model parallelism
n_gpus = [1, 4, 16, 36, 64, 100, 144]
with open('config.json', 'r') as file:
    system = json.load(file)
fig, axs = plt.subplots(1,2,figsize=(10,5), tight_layout=True) 
c1 = 'steelblue'
c2 = 'salmon'
fsz = 18
system['summa_nb'] = 4
lgnd = []
lgnd_tot = []
nvs = 4
system['nvlink_size'] = nvs
lgnd = ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot = ["nvlink{}".format(nvs)]
plot_2d(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="o-")
nvs = 16
system['nvlink_size'] = nvs
lgnd += ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot += ["nvlink{}".format(nvs)]
plot_2d(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="*:")
nvs = 2
system['nvlink_size'] = nvs
lgnd += ["MLP-nvlink{}".format(nvs), "SA-nvlink{}".format(nvs)]
lgnd_tot += ["nvlink{}".format(nvs)]
plot_2d(n_gpus, system, axs, lgnd=lgnd, lgnd_tot=lgnd_tot, lfmt="o--")